In [1]:
from music21 import *
c = converter.parse('mozart.mxl')

threshold = 0.8

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
def compute_similarity(one: stream.Stream, two: stream.Stream, numBars: int = 2, numQuarters: int = 4):
    # check if part was changed for each note
    # check if transposed
    # check if there are added notes
    oneChordify = one.chordify(addPartIdAsGroup=True, removeRedundantPitches=False)
    twoChordify = two.chordify(addPartIdAsGroup=True, removeRedundantPitches=False)
    
    offInt = 4
    total = 0
    intv = 0
    totalSame = 0
    partDiff = False
    
    for i in range(4 * offInt * numBars):
        chord1 = oneChordify.flat.getElementAtOrBefore(i / offInt)
        chord2 = twoChordify.flat.getElementAtOrBefore(i / offInt)
        
        if type(chord1) != chord.Chord:
            continue
        if type(chord2) != chord.Chord:
            continue
        
        notes1 = chord1.notes
        notes2 = chord2.notes
        
        minlen = min(len(notes1), len(notes2))
        
        for j in range(minlen):
            same = True
            transposed = True
            if notes1[j].pitch.ps != notes2[j].pitch.ps:
                same = False
                if intv != 0:
                    if intv != interval.Interval(noteStart=notes1[j], noteEnd=notes2[j]):
                        transposed = False
                else:        
                    intv = interval.Interval(noteStart=notes1[j], noteEnd=notes2[j])
            elif len(notes1[j].groups) > 0 and len(notes2[j].groups) > 0 \
                and notes1[j].groups[0] != notes2[j].groups[0]:
                partDiff = True   
                
            if same or transposed:
                totalSame += 1
            total += 1
            
    return {'s': totalSame / total, 'p': partDiff}

In [3]:
def find_similar(one: stream.Stream, score: stream.Stream, numMeasures, motifIdx, pastChecked, currentMeasure):
    checkedMeasures = [0] * numMeasures
    
    for i in range(currentMeasure + 1, numMeasures - 1):
        if i == 0:
            continue
        
        two = score.measures(i, i+1)
        sim = compute_similarity(one, two, 2, score.getTimeSignatures()[0].barDuration.quarterLength)
         
        if sim['s'] > threshold and pastChecked[i] == 0:
            return i
    
    return -1

In [4]:
def extend(score: stream.Stream, oneNum, twoNum, offsetNum, motifIdx):    
    m1 = score.measure(oneNum + offsetNum)
    m2 = score.measure(twoNum + offsetNum)
    
    sim = compute_similarity(m1, m2, 1, score.getTimeSignatures()[0].barDuration.quarterLength)
    
    if sim['s'] > threshold:
        return True
    
    return False
    
    

In [5]:
def find_motifs_2_bar (score: stream.Stream):
    numMeasures = len(score.parts[0])
    checkedMeasures = [0] * numMeasures
    print("number of measures: ", numMeasures)
    
    motifIdx = 1
    for i in range(0, numMeasures-1):
        if i == 0:
            continue
        print(i)
        if checkedMeasures[i] == 0:
            measureSlice = score.measures(i, i+1)
            motifLoc = find_similar(measureSlice, score, numMeasures, motifIdx, checkedMeasures, i)
            
            offset = 2
            if motifLoc > 0:
                checkedMeasures[motifLoc] = motifIdx
                checkedMeasures[motifLoc+1] = motifIdx
                while extend(score, i, motifLoc, offset, motifIdx):
                    if checkedMeasures[i + offset] != 0 or checkedMeasures[motifLoc + offset] != 0:
                        break
                    checkedMeasures[i + offset] = motifIdx
                    checkedMeasures[motifLoc + offset] = motifIdx
                    offset += 1
                    
            
            checkedMeasures[i] = motifIdx
            checkedMeasures[i+1] = motifIdx
            
            motifIdx += 1
    
    return checkedMeasures

In [6]:
output = find_motifs_2_bar(c)
print(output)

number of measures:  129
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
[0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 6, 6, 5, 5, 7, 7, 8, 8, 8, 8, 9, 9, 10, 10, 11, 11, 6, 6, 12, 12, 13, 13, 14, 14, 14, 15, 15, 14, 14, 16, 16, 17, 17, 17, 18, 18, 18, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 20, 20, 20, 21, 21, 21, 22, 22, 22, 21, 21, 21, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 30, 30, 31, 31]


In [7]:
out = [0, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 6, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 3, 3, 3, 3, 3, 3, 3, 3, 3, 25, 25, 26, 26, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 27, 27, 28, 28, 28, 28, 29, 29, 30, 30, 28, 28, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40]
i = 0
for measure in c.parts[0]:
    if i == 0:
        i += 1
        continue
#     measure[0].lyric = out[i]
    if type(measure) == stream.Measure:
        measure.getElementsByClass(['Note', 'Rest', 'Chord'])[0].lyric = out[i]
#         measure.getElementsByClass(['Note', 'Rest', 'Chord'])[0].lyric = ""
        
    i += 1
    

In [8]:
bounds = [(0, 34), (35, 64), (65, 96), (97, 128)]
segments = [set([])] * len(bounds)
similar = {0:(0,34), 2:(65, 96)}

for i in similar.keys():
    for j in similar.keys():
        if not i < j: continue

        one = similar[i]
        two = similar[j]

        longchunks = []
        currentChunk = out[0]
        length = 0
        for k in range(one[0], one[1]+1):
            if length >= 8:
                longchunks.append(out[k])
            if out[k] == currentChunk:
                length += 1
            else:
                length = 0
                currentChunk = out[k]

        chunknums = set([])
        for k in range(two[0], two[1]+1):
            if out[k] in longchunks:
                chunknums.add(out[k])

        segments[i] = segments[i].union(chunknums)
        segments[j] = segments[j].union(chunknums)

sectionmap = {}
alphabet = "ABCDEFGHJIJKLMNOPQRSTUVWXYZ"
output = ""
iter = 0
for se in segments:
    if len(se) == 0:
        output += alphabet[iter]
        iter += 1
    for elem in se:
        if elem not in sectionmap:
            sectionmap[elem] = alphabet[iter]
            output += alphabet[iter]
            iter += 1
        else:
            output += sectionmap[elem]

print(output)




ABCABD
